# Libraries

In [141]:
# Data manipulation libraries
import pandas as pd # Dataframes

# Statistical libraries
from sklearn.model_selection import train_test_split # Split dataset for validation
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error # Error metrics

# Model libraries
import pickle
from sklearn.linear_model import LinearRegression

# Save Model

In [142]:
def train_model(path: str) -> tuple[LinearRegression, dict[str,float]]:
    df = pd.read_csv(path)\
           .drop(columns = ['Order_ID'])\
           .dropna()

    target_col = 'Delivery_Time_min'
    X = df.drop(columns = target_col)
    y = df[target_col]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size = 0.3, 
        random_state = 42
    )

    model = LinearRegression()
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    error_metrics = {
        'MAPE': mean_absolute_percentage_error(y_test,y_pred),
        'MAE': mean_absolute_error(y_test,y_pred)
    }

    return (model, error_metrics)

In [143]:
model, error_metrics = train_model('data/clean_data_2.csv')
model_scaled, error_metrics_scaled = train_model('data/clean_data_3.csv')

print('Linear Regression Model error metrics')
print(f'MAPE: {error_metrics['MAPE']}')
print(f'MAE: {error_metrics['MAE']}')

Linear Regression Model error metrics
MAPE: 0.09806818588185184
MAE: 5.408311746169921


In [144]:
filename = 'linear_regression_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

print(f"Model saved to {filename}")

Model saved to linear_regression_model.pkl
